# Ensemble Voting Predictor

For each predictor, independently calculate the median value. Choose to vote "up" or "down" depending on the value of each predictor. At the end, assign 0 or less to "approve" and 1 or more to "reject".

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas
import numpy
import matplotlib.pyplot as plt
import itertools
import sklearn
from sklearn.metrics import log_loss

In [4]:
import loan_approval_lib
from loan_approval_lib import logspace, linspace

In [5]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.naive_bayes import CategoricalNB

In [6]:
import itertools

In [7]:
data = loan_approval_lib.load_original_data()

In [8]:
data

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,0.59,Y,3,1
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0.10,N,2,0
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,0.57,N,3,1
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,0.53,N,2,1
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,0.55,Y,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0.11,N,30,0
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0.15,N,19,0
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,0.46,N,28,1
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0.10,N,26,0


# Calculate median for all numerical values

# Calculate median and vote direction for `person_age`

The value of `P` is > 0.5, therefore if `person_age` is above the median value, vote "reject"

In [10]:
data_person_age = data['person_age']
median_person_age = numpy.median(data_person_age)
median_person_age

np.float64(26.0)

In [12]:
data_person_age_2 = data[['person_age', 'loan_status']].copy()
data_person_age_2['person_age_sub_median'] = data_person_age_2['person_age'] - median_person_age
data_person_age_2['person_age_direction'] = (data_person_age_2['person_age_sub_median'] > 0).astype(int)
P_person_age = (data_person_age_2['person_age_direction'] == data_person_age_2['loan_status']).sum() / len(data_person_age_2)
P_person_age

np.float64(0.5166508087535681)

In [15]:
len(
    data[
        (data_person_age_2['person_age_direction'] == 0) &
        (data_person_age_2['loan_status'] == 0)
    ]
)

13777

In [16]:
len(
    data[
        (data_person_age_2['person_age_direction'] == 1) &
        (data_person_age_2['loan_status'] == 1)
    ]
)

3056

In [19]:
13777 + 3056

16833

In [17]:
len(
    data[
        (data_person_age_2['person_age_direction'] == 0) &
        (data_person_age_2['loan_status'] == 1)
    ]
)

4052

In [18]:
len(
    data[
        (data_person_age_2['person_age_direction'] == 1) &
        (data_person_age_2['loan_status'] == 0)
    ]
)

11696

In [20]:
4052 + 11696

15748

# Calculate median and vote direction for `person_income`

The value of `P` is <= 0.5, therefore if `person_income` is below or equal to the median value, vote "reject"

In [23]:
data_person_income = data[['person_income', 'loan_status']].copy()
median_person_income = numpy.median(data_person_income['person_income'])
print(median_person_income)
data_person_income['person_income_sub_median'] = data_person_income['person_income'] - median_person_income
data_person_income['person_income_direction'] = (data_person_income['person_income_sub_median'] > 0).astype(int)
P_person_income = (data_person_income['person_income_direction'] == data_person_income['loan_status']).sum() / len(data_person_income)
print(P_person_income)

55000.0
0.41462815751511617
